<a href="https://colab.research.google.com/github/princejain301/Conversational-Chatbot/blob/main/Dialogue_System_Using_BART_and_Persona_Chat_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This notebook implements a persona-based chatbot using the Hugging Face Transformers library, specifically the BART model.
The model is trained on the Persona-Chat dataset, which contains dialogues where each participant has a defined persona, enabling
the chatbot to generate contextually relevant and personalized responses. Key components include defining hyperparameters for model
configuration, processing dialogue history, and preparing persona statements. The chatbot engages in conversations that reflect
its personality and can adapt to different user inputs. Examples demonstrate how to initialize the chatbot with various personas
and generate responses. This framework can be further extended to explore diverse conversational contexts and enhance user
engagement in dialogue systems.


In [1]:
# Import necessary libraries
from typing import List, TypedDict
from dataclasses import dataclass
from itertools import chain
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# This class holds the configuration settings (hyperparameters) for our chatbot model.

In [2]:
@dataclass
class H2PersonaChatHyperparametersV1:
    """
    This class defines various settings for the chatbot model:
    - model_name: The name of the pre-trained model (e.g., BART).
    - chat_history_pair_length: The number of recent dialogue pairs to consider.
    - persona_max_length: The maximum length of persona descriptions.
    - chat_max_length: The maximum length for chat messages.
    - debug_status: Controls the amount of debug information displayed.
    """
    model_name: str = "facebook/bart-base"
    chat_history_pair_length: int = 7
    persona_max_length: int = 14
    chat_max_length: int = 25
    debug_status: int = 0

# This class represents a sample from our persona chat dataset.

In [3]:
class PersonaChatDatasetSampleV1(TypedDict):
    """
    Represents a single sample from the dataset:
    - persona: A list of statements describing the persona's characteristics.
    - history: A list of previous messages in the conversation.
    - sample_id: A unique identifier for this sample.
    """
    persona: List[str]
    history: List[str]
    sample_id: str

# This class defines the structure for the input data sent to the model.

In [4]:
class H2Seq2SeqInferenceSampleDictV1(TypedDict):
    """Contains the input IDs and attention mask for the model."""
    input_ids: List[int]
    attention_mask: List[int]

# This class uses PyTorch tensors for the input data structure.

In [5]:
class H2Seq2SeqInferenceSampleDictV2(TypedDict):
    """Similar to the previous class but uses tensors instead of lists."""
    input_ids: torch.Tensor
    attention_mask: torch.Tensor

# Function to flatten a list of lists into a single list.

In [6]:
def flat_list(list_of_lists: List[List]) -> List:
    """Flattens a list of lists into a single list."""
    return list(chain.from_iterable(list_of_lists))

# Class to prepare input samples for the Seq2Seq model.

In [8]:
class H2Seq2SeqInferencePersonaSampleV1:
    def __init__(self, dataset_sample: PersonaChatDatasetSampleV1, tokenizer: AutoTokenizer, hyperparameters: H2PersonaChatHyperparametersV1) -> None:
        """Initializes the input sample with the dataset, tokenizer, and hyperparameters."""
        self.dataset_sample = dataset_sample  # Stores the dataset sample
        self.tokenizer = tokenizer  # Tokenizer for converting text to model-readable format
        self.hyperparameters = hyperparameters  # Stores model settings

    def add_spaces_after(self, items: List[str]) -> List[str]:
        """Adds a space after each item in a list of strings."""
        return [item + " " for item in items]

    @property
    def bos_token_id(self):
        """Returns the beginning-of-sequence token ID, if applicable."""
        if "t5" in self.hyperparameters.model_name:
            return []
        return [self.tokenizer.bos_token_id] if self.tokenizer.bos_token_id else []

    @property
    def eos_token_id(self):
        """Returns the end-of-sequence token ID, if applicable."""
        return [self.tokenizer.eos_token_id] if self.tokenizer.eos_token_id else []

    def add_sep_beetween(self, items: List[str], sep=" EOS ") -> List[str]:
        """Adds a separator between items in a list."""
        for i in range(1, len(items)):
            items[i] = sep + items[i]  # Adds the separator before each item except the first
        return items

    def add_spaces_between(self, items: List[str]) -> List[str]:
        """Adds spaces after items and removes trailing spaces from the last item."""
        items = self.add_spaces_after(items)
        items[-1] = items[-1].strip()  # Ensures no extra space at the end
        return items

    def get_sample(self) -> H2Seq2SeqInferenceSampleDictV1:
        """Prepares the input sample for the model."""
        # Get recent dialog history
        dialog_history = self.dataset_sample["history"][-self.hyperparameters.chat_history_pair_length * 2 - 1:]
        dialog_history = self.add_sep_beetween(dialog_history)  # Add separators between messages

        # Get persona statements and add separators
        persona = self.dataset_sample["persona"]
        persona = self.add_sep_beetween(persona, sep=" ")

        # Encode knowledge and context tokens
        KNOWLEDGE_IDS = self.tokenizer.encode(" [KNOWLEDGE] ", add_special_tokens=False)
        CONTEXT_IDS = self.tokenizer.encode(" [CONTEXT] ", add_special_tokens=False)

        # Encode the chat history and persona facts into token IDs
        encoded_history = self.tokenizer.batch_encode_plus(dialog_history, add_special_tokens=False, truncation=True, max_length=self.hyperparameters.chat_max_length)
        encoded_history = flat_list(encoded_history["input_ids"])

        encoded_persona = self.tokenizer.batch_encode_plus(persona, add_special_tokens=False, truncation=True, max_length=self.hyperparameters.persona_max_length)
        encoded_persona = flat_list(encoded_persona["input_ids"])

        # Combine all encoded parts into a single input
        input_ids = [
            *self.bos_token_id,  # Beginning of sequence token (if any)
            *CONTEXT_IDS,  # Context token
            *encoded_history,  # Encoded history
            *KNOWLEDGE_IDS,  # Knowledge token
            *encoded_persona,  # Encoded persona
            *self.eos_token_id,  # End of sequence token (if any)
        ]
        # Create an attention mask (1s for all tokens in input_ids)
        attention_mask = [1] * len(input_ids)

        # Return the prepared sample for inference
        return H2Seq2SeqInferenceSampleDictV1(input_ids=input_ids, attention_mask=attention_mask)


# This class represents the chatbot itself.

In [9]:
class DialogBotV1:
    def __init__(self, model: AutoModelForSeq2SeqLM, tokenizer: AutoTokenizer, hyperparameters: H2PersonaChatHyperparametersV1, history: List[str] = None, persona: List[str] = None, device: str = "cuda", shuffle_persona: bool = True):
        """Initializes the chatbot with the model, tokenizer, hyperparameters, chat history, and persona."""
        self.model = model  # The pre-trained model for generating responses
        self.tokenizer = tokenizer  # Tokenizer for text processing
        self.hyperparameters = hyperparameters  # Model settings
        self.device = device  # Device for running the model (GPU or CPU)
        self.shuffle_persona = shuffle_persona  # Option to shuffle persona facts

        self.debug_status = hyperparameters.debug_status  # For debug information

        # Initialize chat history and persona
        self.history = history if history is not None else []
        self.persona = persona if persona is not None else []

    def _get_sample(self, persona: List[str], history: List[str]) -> H2Seq2SeqInferenceSampleDictV1:
        """Prepares a sample for the model based on the current persona and history."""
        # Create a dataset sample
        dataset_sample = PersonaChatDatasetSampleV1(persona=persona, history=history)
        sample = H2Seq2SeqInferencePersonaSampleV1(tokenizer=self.tokenizer, hyperparameters=self.hyperparameters, dataset_sample=dataset_sample)  # Prepare sample
        sample = sample.get_sample()  # Get the formatted sample
        print(self.tokenizer.decode(sample['input_ids']))  # Decode input IDs to check what will be sent to the model

        # Convert the input and attention mask to PyTorch tensors for the model
        for key in sample.keys():
            sample[key] = torch.tensor(sample[key]).unsqueeze(0).to(self.device)  # Unsqueeze to add a batch dimension

        return sample

    def next_response(self, **generation_params) -> str:
        """Generates the next response from the chatbot."""
        # Prepare a sample from the current persona and history
        sample = self._get_sample(persona=self.persona, history=self.history)
        # Generate a response using the model
        answer = self.generate_response(sample, **generation_params)
        answer = self.tokenizer.batch_decode(answer, skip_special_tokens=True)  # Decode the response to readable text
        self.history.append(answer[0])  # Add the response to the chat history
        return answer[0]  # Return the generated response

    def generate_response(self, sample: H2Seq2SeqInferenceSampleDictV1, **generation_params):
        """
        Generates a response based on the provided sample.
        generation_params: Additional parameters to customize response generation (like max tokens).
        """
        with torch.no_grad():  # Disable gradient calculations for inference (saves memory)
            return self.model.generate(**sample, **generation_params)  # Call the model to generate a response

# Load the pre-trained model and tokenizer

In [10]:
# Load the pre-trained model and tokenizer
PRETRAINED_MODEL_NAME_OR_PATH = "DeepPavlov/bart-base-en-persona-chat"
PAIR_DIALOG_HISTORY_LENGTH = 2  # How many pairs of dialogue to consider

# Maximum length for chat messages and persona statements in tokens
CHAT_MAX_LENGTH = 20
PERSONA_MAX_LENGTH = 10

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/279M [00:00<?, ?B/s]

# Set hyperparameters for the chatbot

In [17]:
if torch.cuda.is_available():
    model.half()

hyperparameters = H2PersonaChatHyperparametersV1(
    chat_history_pair_length=PAIR_DIALOG_HISTORY_LENGTH,
    persona_max_length=PERSONA_MAX_LENGTH,
    chat_max_length=CHAT_MAX_LENGTH,
    model_name=PRETRAINED_MODEL_NAME_OR_PATH,
)


This code snippet prepares a pre-trained model for a chatbot that uses persona-based dialogue. It defines the model's name and the lengths for chat messages and persona statements. It checks whether a GPU is available and sets the device accordingly, allowing for faster computation. The model is then loaded from a specified path and transferred to the chosen device. Finally, it is set to evaluation mode to ensure consistent behavior during response generation.

In [22]:
# Specify the name or path of the pre-trained model to be used for persona-based chat.
PRETRAINED_MODEL_NAME_OR_PATH = "DeepPavlov/bart-base-en-persona-chat"

# Set the number of pairs of dialogue history to consider for context in conversations.
PAIR_DIALOG_HISTORY_LENGTH = 2

# Maximum token length for a single chat message (sentence).
CHAT_MAX_LENGTH = 25

# Maximum token length for a single persona statement (sentence).
PERSONA_MAX_LENGTH = 19

# Determine the device to use for model inference: CUDA (GPU) if available, otherwise CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained Seq2Seq model for generating responses and transfer it to the selected device.
model = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)

# Move the model to the appropriate device (GPU or CPU).
model.to(device)

# Set the model to evaluation mode to disable dropout and ensure deterministic behavior during inference.
model.eval()


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

# Create an instance of the chatbot

In [15]:
dialog_bot = DialogBotV1(model=model, tokenizer=tokenizer, hyperparameters=hyperparams)



This code snippet demonstrates how to interact with the persona-based chatbot model, `DialogBotV1`, using a specified persona and chat history. Here's a breakdown of the key components:

1. **Persona Definition**:
   - `persona`: This list includes two statements that describe the chatbot's personality: it likes to play guitar and dislikes onions. These traits help the model generate responses that are consistent with this persona.

2. **Dialogue History**:
   - `history`: This list contains the previous message in the conversation, where the user expresses a dislike for discussing politics and prompts the chatbot for its opinion.

3. **Chatbot Initialization**:
   - `persona_bot`: An instance of the `DialogBotV1` class is created, which initializes the chatbot with the pre-trained model, tokenizer, hyperparameters, chat history, and persona. The `device` variable specifies whether the model runs on a GPU or CPU.

4. **Response Generation Parameters**:
   - `GENERATION_PARAMS`: This dictionary specifies parameters for generating the response. It limits the response to a maximum of 60 new tokens, applies a penalty to reduce repetitive responses, and restricts the number of candidate responses to the top 10.

5. **Generating a Response**:
   - `response`: The `next_response` method is called on the chatbot instance, using the defined generation parameters. This method generates a response based on the provided chat history and persona.

6. **Output**:
   - The response is printed, showcasing how the chatbot, reflecting its persona, shifts the conversation away from politics to music, saying, "I am not into politics. I am into music." This illustrates the chatbot's ability to engage in personalized and contextually relevant dialogue.



In [20]:
persona = [
    "I like to play guitar.",
    "I hate onions."
]

history = [
    "I hate to talk about politics, what about you?"
]

persona_bot = DialogBotV1(
        model=model,
        tokenizer=tokenizer,
        hyperparameters=hyperparameters,
        history=history,
        persona=persona,
        device=device,
    )

GENERATION_PARAMS = {
    "max_new_tokens": 60,
    "penalty_alpha": 0.15,
    "top_k": 10
}
response = persona_bot.next_response(
    **GENERATION_PARAMS,
)

print(response)
# i am not into politics. i am into music.

 [CONTEXT] I hate to talk about politics, what about you? [KNOWLEDGE] I like to play guitar. I hate onions.</s>
i don't like to talk about politics.


In [21]:
response

"i don't like to talk about politics."

In [23]:
# Define a different persona for the chatbot
persona = [
    "I love hiking in the mountains.",
    "I enjoy cooking Italian food."
]

# Set the dialogue history
history = [
    "What do you think about outdoor activities?"
]

# Create an instance of the chatbot with the new persona and history
persona_bot = DialogBotV1(
    model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=history,
    persona=persona,
    device=device,
)

# Set response generation parameters
GENERATION_PARAMS = {
    "max_new_tokens": 50,
    "penalty_alpha": 0.1,
    "top_k": 5
}

# Generate a response
response = persona_bot.next_response(**GENERATION_PARAMS)

# Print the response
print(response)
# Example output: "I absolutely love hiking! The mountains are so refreshing."


 [CONTEXT] What do you think about outdoor activities? [KNOWLEDGE] I love hiking in the mountains. I enjoy cooking Italian food.</s>
i love hiking in the mountains


In [25]:
# Define another persona for the chatbot
persona = [
    "I'm a big fan of science fiction movies.",
    "I love reading fantasy novels."
]

# Set the dialogue history
history = [
    "Do you like watching movies or reading books more?"
]

# Create an instance of the chatbot with the new persona and history
persona_bot = DialogBotV1(
    model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=history,
    persona=persona,
    device=device,
)

# Set response generation parameters
GENERATION_PARAMS = {
    "max_new_tokens": 70,
    "penalty_alpha": 0.2,
    "top_k": 7
}

# Generate a response
response = persona_bot.next_response(**GENERATION_PARAMS)

# Print the response
print(response)
# Example output: "That's a tough choice! I love both, but if I had to pick, I’d go for sci-fi movies."


 [CONTEXT] Do you like watching movies or reading books more? [KNOWLEDGE] I'm a big fan of science fiction movies. I love reading fantasy novels.</s>
i love science fiction movies and fantasy novels


In [24]:
response

'i love hiking in the mountains'

In [27]:
from typing import List, TypedDict
from dataclasses import dataclass
from itertools import chain
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

@dataclass
class H2PersonaChatHyperparametersV1:
    model_name: str = "facebook/bart-base"
    chat_history_pair_length: int = 7
    persona_max_length: int = 14
    chat_max_length: int = 25
    debug_status: int = 0

class PersonaChatDatasetSampleV1(TypedDict):
    persona: List[str]
    history: List[str]
    sample_id: str

class H2Seq2SeqInferenceSampleDictV1(TypedDict):
    input_ids: List[int]
    attention_mask: List[int]

class SemiAutonomousDialogBot:
    def __init__(
        self,
        model: AutoModelForSeq2SeqLM,
        tokenizer: AutoTokenizer,
        hyperparameters: H2PersonaChatHyperparametersV1,
        history: List[str] = None,
        persona: List[str] = None,
        device: str = "cuda",
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.hyperparameters = hyperparameters
        self.device = device
        self.history = history if history else []
        self.persona = persona if persona else []

    def _get_sample(self, persona: List[str], history: List[str]) -> H2Seq2SeqInferenceSampleDictV1:
        dataset_sample = PersonaChatDatasetSampleV1(persona=persona, history=history)
        dialog_history = history[-self.hyperparameters.chat_history_pair_length * 2 - 1:]

        # Combine history and persona
        combined_input = persona + dialog_history
        encoded = self.tokenizer(
            combined_input,
            return_tensors="pt",
            padding=True,  # Enable padding
            truncation=True,
            max_length=self.hyperparameters.chat_max_length
        )

        input_ids = encoded["input_ids"].to(self.device)
        attention_mask = encoded["attention_mask"].to(self.device)

        return H2Seq2SeqInferenceSampleDictV1(input_ids=input_ids, attention_mask=attention_mask)

    def next_response(self) -> str:
        sample = self._get_sample(persona=self.persona, history=self.history)
        with torch.no_grad():
            output = self.model.generate(**sample)

        answer = self.tokenizer.decode(output[0], skip_special_tokens=True)

        # Check if human input is needed
        if self.needs_human_input(answer):
            return f"{answer} Does that sound good to you?"

        self.history.append(answer)
        return answer

    def needs_human_input(self, response: str) -> bool:
        return "politics" in response or "I'm not sure" in response

# Set up the model and tokenizer
PRETRAINED_MODEL_NAME_OR_PATH = "DeepPavlov/bart-base-en-persona-chat"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH).to(device)
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)

hyperparameters = H2PersonaChatHyperparametersV1()

# Define persona and history
persona = [
    "I like to play guitar.",
    "I hate onions."
]

history = [
    "I hate to talk about politics, what about you?"
]

# Create an instance of the semi-autonomous dialog bot
persona_bot = SemiAutonomousDialogBot(
    model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=history,
    persona=persona,
    device=device,
)

# Generate a response
response = persona_bot.next_response()
print(response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


i love to play the guitar


In [28]:
response

'i love to play the guitar'

In [29]:
import random

class SemiAutonomousDialogBot(DialogBotV1):
    def __init__(self, model, tokenizer, hyperparameters, history=None, persona=None, device="cuda"):
        super().__init__(model, tokenizer, hyperparameters, history, persona, device)

    def needs_human_input(self, response: str) -> bool:
        """
        Check if the response contains sensitive topics that may require user intervention.
        """
        sensitive_topics = ["politics", "confusion", "hate", "fear"]
        return any(topic in response.lower() for topic in sensitive_topics)

    def next_response(self, **generation_params) -> str:
        """
        Override to check for human intervention.
        """
        sample = self._get_sample(persona=self.persona, history=self.history)
        answer = self.generate_response(sample, **generation_params)
        answer = self.tokenizer.batch_decode(answer, skip_special_tokens=True)[0]

        # Check if intervention is needed
        if self.needs_human_input(answer):
            intervention_prompt = f"{answer} Does that sound good to you?"
            self.history.append(intervention_prompt)
            return intervention_prompt

        self.history.append(answer)
        return answer

# Example usage
persona = [
    "I like to play guitar.",
    "I hate onions."
]

history = [
    "I think politics is so confusing. What do you think about political debates?"
]

persona_bot = SemiAutonomousDialogBot(
    model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=history,
    persona=persona,
    device=device,
)

GENERATION_PARAMS = {
    "max_new_tokens": 60,
    "penalty_alpha": 0.15,
    "top_k": 10
}

response = persona_bot.next_response(**GENERATION_PARAMS)
print(response)  # Example output: "I don't really enjoy talking about politics. Sometimes I feel confused about the issues too. Does that sound good to you?"


 [CONTEXT] I think politics is so confusing. What do you think about political debates? [KNOWLEDGE] I like to play guitar. I hate onions.</s>
i do not really care about politics. Does that sound good to you?


In [35]:
!pip install langchain

In [34]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00


In [36]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFaceLLM
from langchain.chains import ConversationalChain

# Define the model and its parameters
model_name = "facebook/bart-base"
llm = HuggingFaceLLM(model_name=model_name)

# Define a prompt template for generating responses
prompt_template = PromptTemplate(
    input_variables=["context", "knowledge", "user_input"],
    template="""[CONTEXT] {context}
[KNOWLEDGE] {knowledge}
User: {user_input}
Chatbot:""",
)

# Create an LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Define a function to generate a response and check for human intervention
def get_response(context, knowledge, user_input):
    # Generate the response using the LLM chain
    response = llm_chain.run(context=context, knowledge=knowledge, user_input=user_input)

    # Check if human intervention is needed
    if "confusing" in user_input or "overwhelmed" in user_input:
        return f"{response} Does that sound good to you?"
    else:
        return response

# Define persona and initial conversation
persona_knowledge = "I enjoy reading sci-fi novels. I'm a vegetarian."
context = "I'm here to discuss various topics."

# Example user input
user_input = "What do you think about science fiction books?"

# Get the response
response = get_response(context, persona_knowledge, user_input)

print(response)


ImportError: cannot import name 'HuggingFaceLLM' from 'langchain.llms' (/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py)